In [27]:
# from PIL import Image
# from numpy import asarray
# import numpy as np

# # load the image
# image = Image.open('HA_3.png')
# # convert image to numpy array
# data = asarray(image)
# print(type(data))
# # summarize shape
# print(data.shape)

# # create Pillow image
# image2 = Image.fromarray(data)
# print(type(image2))

# # summarize image details
# print(image2.mode)
# print(image2.size)

# im = np.array(Image.open('HA_3.png').convert('L').resize((30,30))) #you can pass multiple arguments in single line
# print(type(im))

# Image.fromarray(im).save('HA_3_transformed.png')

# print(im)

import cv2 as cv
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import PIL.ImageOps
import os
import glob
from sklearn.model_selection import train_test_split

def resize_only(file_name):
	img = Image.open(file_name).convert('LA')
	# img = img.filter(ImageFilter.SMOOTH)
	# img = img.filter(ImageFilter.SHARPEN)
	img = img.resize((28, 28))
	# img = PIL.ImageOps.invert(img)
	img.save("temp.png")
	img = cv.imread('temp.png', 0)
	img = cv.bitwise_not(img)
	img =  Image.fromarray(img)
	
	return img

def resize(file_name, invert):
	# image = cv.imread('%s' % file_name,0)
	img = Image.open(file_name).convert('LA')
# 	if invert==True:
# 		img = PIL.ImageOps.invert(img)
	enhancer = ImageEnhance.Contrast(img)
	
	factor = 2.0
	img = enhancer.enhance(factor)
	img = img.filter(ImageFilter.SMOOTH)
	img = img.filter(ImageFilter.SHARPEN)
	img = img.resize((28, 28))
	# print(type(img))    
    
	img.save("temp.png")
    
	img = cv.imread('temp.png', 0)
    
	if invert==True:
		img = cv.bitwise_not(img)
	ret,th = cv.threshold(img,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
	blur = cv.GaussianBlur(img,(5,5),0)
	ret2,th2 = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
	mask = np.where(th ==  0, th2, 255)
    
	img = np.where(mask == 0, img, 255)

    # create a CLAHE object (Arguments are optional).
	clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
	img = clahe.apply(img)

	_,img_remove = cv.threshold(img,95,255,cv.THRESH_TOZERO)
	img_remove = cv.bitwise_not(img_remove)
	_,img = cv.threshold(img_remove,95,255,cv.THRESH_TOZERO)

	# img = cv.bitwise_not(img)
    
	img =  Image.fromarray(img)
    
	# print(img)
	# pix = np.array(img)
	# print(type(pix))
	# print(pix.shape)
	# print(pix)
	# pix = pix[:, :, 0]
	# print(type(pix))
	# print(pix.shape)
	# print(pix)
	return img
	# img.save("resized_"+file_name)

def preprocess():
	# crop(str(sys.argv))
	script_dir = os.path.abspath('')
	i=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
	j=[30,47,19,67,37,27,16,21,60,60,56,120,25,14,61,42,63,60,23,35,36,84,56,90,78,18,24,22]
	k=[12,30,7,45,16,11,7,9,40,25,24,80,10,6,39,28,42,40,10,24,24,36,36,60,52,7,10,9]
	ganed = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]

	final_np = np.array([])
	final_label = np.array([])

    #   train_image to ready_to_train
	for x in range(0, len(i), 1):
		sumImg = 0
		train_directory_path = os.path.join(script_dir,"train_image")
		train_count_image = len(glob.glob1(train_directory_path,"%s_*.png" % (i[x])))

		for y in range(1, train_count_image+1):
			rel_path = "train_image/%s_%s.png" % (i[x],str(y))
			abs_file_path = os.path.join(script_dir, rel_path)
            
			img = resize_only(abs_file_path)
			img.save("train-test_image/%s_%s.png" % (i[x],str(y)))
			# img = resize(abs_file_path, invert=False)
			# img.save("ready_to_train_3/%s_%s.png" % (i[x],str(y)))
		sumImg = sumImg + train_count_image

		test_directory_path = os.path.join(script_dir,"test_image/%s" % i[x])
		test_count_image = len(glob.glob1(test_directory_path,"%s_*.png" % (i[x])))

		for y in range(1, test_count_image+1):
			rel_path = os.path.join(test_directory_path,"%s_%s.png" % (i[x],str(y)))
			abs_file_path = os.path.join(script_dir, rel_path)
            
			img = resize_only(abs_file_path)
			img.save("train-test_image/%s_%s.png" % (i[x],str(y+sumImg)))
		sumImg = sumImg + test_count_image

		if x == 0:
			for y in range(1, sumImg+1):
				rel_path = os.path.join("train-test_image","%s_%s.png" % (i[x],str(y)))
				abs_file_path = os.path.join(script_dir, rel_path)

				img = Image.open(abs_file_path)
				img = np.array(img)
				img = img[:, :]
				final_np = np.append(final_np,img)
				final_label = np.append(final_label,x)
			final_np = final_np.reshape((sumImg, 28, 28, 1)).astype(np.float32)
			final_label = final_label.reshape((sumImg)).astype(np.int32)
			data_train, data_test, labels_train, labels_test = train_test_split(final_np, final_label, test_size=0.30, random_state=42)
			print(final_label)
			print(data_train.shape)
			print(data_test.shape)
			print(labels_train)
			print(labels_test)
# #   GAN_image to ready_to_train
# 	for x in ganed:
# 		for y in range(1, 26):
# 			rel_path = "GAN_generated_images/%s_%s.png" % (i[x],str(y+j[x]))
# # 			rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)
            
# 			img = resize(abs_file_path, invert=True)
# 			img.save("ready_to_train_3/%s_%s.png" % (i[x],str(y+j[x])))

# #   test_image to ready_to_test (validation)
# 	sumImg = 0
# 	final_np = np.array([])
# 	final_label = np.array([])
# 	for x in range(0, len(i), 1):
# 		sumImg = sumImg + k[x]
# 		for y in range(1, int((k[x])/2)+1):
# 			rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			# rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)

# 			# img = resize(abs_file_path, invert=False)
# 			# img.save("ready_to_test_2/%s_%s.png" % (i[x],str(y)))
# 			img = resize_only(abs_file_path)
# 			img.save("ready_to_test/%s_%s.png" % (i[x],str(y)))

# # 	test_image to ready_to_test (test)
# 	sumImg = 0
# 	for x in range(0, len(i), 1):
# 		sumImg = sumImg - (-k[x]//2)
# 		for y in range((-(-k[x])//2)+1, k[x]+1):
# #             rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			rel_path = "test_image/%s/%s_%s.png" % (i[x],i[x],str(y))
# 			# rel_path = "ready_to_train/%s_%s.png" % (i[x],str(y))
# 			abs_file_path = os.path.join(script_dir, rel_path)
            
# 			img = Image.open(abs_file_path)
# 			img = np.array(img)
# 			img = img[:, :, 0]
# 			img = resize_only(abs_file_path)
# 			img.save("ready_to_test/%s_%s.png" % (i[x],str(y)))
# 			# img = resize(abs_file_path, invert=False)
# 			# img.save("ready_to_test_2/%s_%s.png" % (i[x],str(y)))

# if __name__ == "__main__":
# 	main()



[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
(29, 28, 28, 1)
(13, 28, 28, 1)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]


In [1]:
def get_sunda(aksara):
    script_dir = os.path.abspath('')
    i=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
    j=[30,47,19,67,37,27,16,21,60,60,56,120,25,14,61,42,63,60,23,35,36,84,56,90,78,18,24,22]
    k=[12,30,7,45,16,11,7,9,40,25,24,80,10,6,39,28,42,40,10,24,24,36,36,60,52,7,10,9]
    ganed = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]

    final_np = np.array([])
    final_label = np.array([])

    #   train_image to ready_to_train
#for x in range(0, len(i), 1):
    sumImg = 0
    directory_path = os.path.join(script_dir,"train-test_image")
    count_image = len(glob.glob1(directory_path,"%s_*.png" % (i[aksara])))

    for y in range(1, count_image+1):
        abs_file_path = os.path.join(directory_path, "%s_%s.png" % (i[aksara],str(y)))
        img = Image.open(abs_file_path)
        img = np.array(img)
        img = img[:, :]
        final_np = np.append(final_np,img)
        final_label = np.append(final_label,aksara)
    sumImg = sumImg + count_image
    final_np = final_np.reshape((sumImg, 28, 28, 1)).astype(np.float32)
    final_label = final_label.reshape((sumImg)).astype(np.int32)
    data_train, data_test, labels_train, labels_test = train_test_split(final_np, final_label, test_size=0.30, random_state=42)
    return(data_train,sumImg)
#     print(final_label)
#     print(data_train.shape)
#     print(data_test.shape)
#     print(labels_train)
#     print(labels_test)

In [2]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np
import cv2 as cv
import os
import scipy
import tensorflow as tf
from PIL import Image
import glob
from sklearn.model_selection import train_test_split

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, aksara, aksara_num, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
#         preprocess()
        X_train, sumTrain= get_sunda(aksara)
        X_train = X_train[:,:,:,0]
#         (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        
        min_loss = 100

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
#             print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                print ("%d-%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (aksara, epoch, d_loss[0], 100*d_loss[1], g_loss))
                self.sample_images(epoch, aksara, sumTrain, aksara_num)
            
#             if d_loss[0] + g_loss < min_loss and epoch > 10000:
#                 min_loss = d_loss[0] + g_loss
#                 self.sample_images(epoch, aksara, sumTrain, aksara_num)

    def sample_images(self, epoch, aksara, sumTrain, aksara_num):
        aksaraList=["A","BA","CA","DA","GA","HA","I","JA","KA","LA","MA","NA","NGA","NYA","PA","PANELENG","PANEULEUNG","PANGHULU","PANGLAYAR","PANOLONG","PANYUKU","PATEN","RA","SA","TA","U","WA","YA"]
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        for i in range(0,aksara_num):
            img = gen_imgs[i,:,:,0]
            normalizedImg = np.zeros((28,28))
            normalizedImg = cv.normalize(img, normalizedImg, 0, 255, cv.NORM_MINMAX)
            cv.imwrite("GAN_generated_images/%s_%s.png" % (aksaraList[aksara],str(sumTrain+i+1)), normalizedImg)
        for i in range(0,4):
            img = gen_imgs[i,:,:,0]
            normalizedImg = np.zeros((28,28))
            normalizedImg = cv.normalize(img, normalizedImg, 0, 255, cv.NORM_MINMAX)
            cv.imwrite("GAN_generated_images/%s_Cadangan_%s.png" % (aksaraList[aksara],str(i+1)), normalizedImg)

if __name__ == '__main__':
#     %%time
    aksara=[26,27]
    aksara_num=[22,24]
    for i in range(0,2):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
#     a = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]
#     for i in a:
#         gan.train(aksara=i, epochs=30001, batch_size=32, sample_interval=2000)

Using TensorFlow backend.
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505


C:\Users\adjie\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


26-0 [D loss: 0.742610, acc.: 43.75%] [G loss: 0.857420]
26-1000 [D loss: 0.962812, acc.: 35.42%] [G loss: 3.058529]
26-2000 [D loss: 0.734340, acc.: 51.04%] [G loss: 1.886794]
26-3000 [D loss: 0.317370, acc.: 85.42%] [G loss: 3.307196]
26-4000 [D loss: 0.178927, acc.: 93.75%] [G loss: 3.203070]
26-5000 [D loss: 0.098615, acc.: 96.88%] [G loss: 5.780303]
26-6000 [D loss: 0.169417, acc.: 94.79%] [G loss: 3.939962]
26-7000 [D loss: 0.104205, acc.: 95.83%] [G loss: 3.565294]
26-8000 [D loss: 0.191808, acc.: 93.75%] [G loss: 3.142579]
26-9000 [D loss: 0.260847, acc.: 87.50%] [G loss: 2.447149]
26-10000 [D loss: 0.265107, acc.: 90.62%] [G loss: 3.378236]
26-11000 [D loss: 0.133662, acc.: 95.83%] [G loss: 3.884812]
26-12000 [D loss: 0.288578, acc.: 89.58%] [G loss: 2.994449]
26-13000 [D loss: 0.174763, acc.: 93.75%] [G loss: 4.721326]
26-14000 [D loss: 0.161531, acc.: 95.83%] [G loss: 4.100693]
26-15000 [D loss: 0.222556, acc.: 91.67%] [G loss: 4.909410]
26-16000 [D loss: 0.203225, acc.: 92.

In [2]:

if __name__ == '__main__':
#     %%time
    aksara=[18,25]
    aksara_num=[22,24]
    for i in range(0,2):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
#     a = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]
#     for i in a:
#         gan.train(aksara=i, epochs=30001, batch_size=32, sample_interval=2000)

Using TensorFlow backend.
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505


C:\Users\adjie\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


18-0 [D loss: 0.639359, acc.: 51.04%] [G loss: 0.500560]
18-1000 [D loss: 0.636255, acc.: 55.21%] [G loss: 1.093661]
18-2000 [D loss: 0.293562, acc.: 78.12%] [G loss: 4.345566]
18-3000 [D loss: 0.296327, acc.: 85.42%] [G loss: 2.882236]
18-4000 [D loss: 0.334307, acc.: 84.38%] [G loss: 2.349632]
18-5000 [D loss: 0.295570, acc.: 85.42%] [G loss: 2.278278]
18-6000 [D loss: 0.423810, acc.: 78.12%] [G loss: 2.168582]
18-7000 [D loss: 0.324320, acc.: 83.33%] [G loss: 2.087028]
18-8000 [D loss: 0.280282, acc.: 88.54%] [G loss: 2.602711]
18-9000 [D loss: 0.372309, acc.: 83.33%] [G loss: 2.500665]
18-10000 [D loss: 0.558994, acc.: 70.83%] [G loss: 2.409971]
18-11000 [D loss: 0.378187, acc.: 81.25%] [G loss: 2.509325]
18-12000 [D loss: 0.294023, acc.: 85.42%] [G loss: 2.347069]
18-13000 [D loss: 0.297227, acc.: 83.33%] [G loss: 2.964416]
18-14000 [D loss: 0.295527, acc.: 86.46%] [G loss: 2.514031]
18-15000 [D loss: 0.317625, acc.: 85.42%] [G loss: 2.954273]
18-16000 [D loss: 0.498343, acc.: 80.

In [2]:

if __name__ == '__main__':
#     %%time
    aksara=[7,12,13]
    aksara_num=[24,21,24]
    for i in range(0,4):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
#     a = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]
#     for i in a:
#         gan.train(aksara=i, epochs=30001, batch_size=32, sample_interval=2000)

Using TensorFlow backend.
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\adjie\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505


C:\Users\adjie\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


7-0 [D loss: 0.706975, acc.: 46.88%] [G loss: 1.057230]
7-1000 [D loss: 0.528538, acc.: 64.58%] [G loss: 2.225678]
7-2000 [D loss: 0.333550, acc.: 92.71%] [G loss: 3.261159]
7-3000 [D loss: 0.145060, acc.: 96.88%] [G loss: 6.615382]
7-4000 [D loss: 0.246441, acc.: 85.42%] [G loss: 3.124181]
7-5000 [D loss: 0.205465, acc.: 94.79%] [G loss: 3.377589]
7-6000 [D loss: 0.100408, acc.: 97.92%] [G loss: 4.695069]
7-7000 [D loss: 0.310659, acc.: 90.62%] [G loss: 3.413656]
7-8000 [D loss: 0.172079, acc.: 92.71%] [G loss: 3.605320]
7-9000 [D loss: 0.317906, acc.: 89.58%] [G loss: 3.798021]
7-10000 [D loss: 0.157956, acc.: 93.75%] [G loss: 5.252641]
7-11000 [D loss: 0.272350, acc.: 90.62%] [G loss: 3.552936]
7-12000 [D loss: 0.307636, acc.: 84.38%] [G loss: 3.139872]
7-13000 [D loss: 0.220902, acc.: 92.71%] [G loss: 4.544974]
7-14000 [D loss: 0.337134, acc.: 85.42%] [G loss: 2.883662]
7-15000 [D loss: 0.265410, acc.: 91.67%] [G loss: 2.349604]
7-16000 [D loss: 0.220954, acc.: 94.79%] [G loss: 3.1

IndexError: list index out of range

In [6]:
if __name__ == '__main__':
#     %%time
    aksara=[2,4,5,6]
    aksara_num=[24,8,19,24]
    for i in range(0,4):
        gan = GAN()
        gan.train(aksara[i], aksara_num[i], epochs=30001, batch_size=48, sample_interval=1000)
        print("-----------------------------------------------------------------")
#     a = [0,2,4,5,6,7,12,13,18,19,20,25,26,27]
#     for i in a:
#         gan.train(aksara=i, epochs=30001, batch_size=32, sample_interval=2000)

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505


C:\Users\adjie\anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


2-0 [D loss: 0.709409, acc.: 56.25%] [G loss: 0.949301]
2-500 [D loss: 0.215756, acc.: 95.83%] [G loss: 4.677984]
2-1000 [D loss: 0.509335, acc.: 67.71%] [G loss: 2.536737]
2-1500 [D loss: 0.377938, acc.: 88.54%] [G loss: 4.425233]
2-2000 [D loss: 0.256330, acc.: 92.71%] [G loss: 3.002691]
2-2500 [D loss: 0.342475, acc.: 91.67%] [G loss: 2.451769]
2-3000 [D loss: 0.192805, acc.: 94.79%] [G loss: 3.565622]
2-3500 [D loss: 0.483268, acc.: 80.21%] [G loss: 3.228659]
2-4000 [D loss: 0.323413, acc.: 88.54%] [G loss: 2.721824]
2-4500 [D loss: 0.099963, acc.: 98.96%] [G loss: 3.551860]
2-5000 [D loss: 0.150127, acc.: 94.79%] [G loss: 2.973430]
2-5500 [D loss: 0.280885, acc.: 88.54%] [G loss: 3.068622]
2-6000 [D loss: 0.226558, acc.: 90.62%] [G loss: 2.634293]
2-6500 [D loss: 0.246897, acc.: 88.54%] [G loss: 3.175595]
2-7000 [D loss: 0.318792, acc.: 85.42%] [G loss: 2.419063]
2-7500 [D loss: 0.300048, acc.: 88.54%] [G loss: 2.658532]
2-8000 [D loss: 0.298076, acc.: 84.38%] [G loss: 3.140390]
2

In [ ]:
%%time
aksara=2
aksara_num=27
gan = GAN()
gan.train(aksara, aksara_num, epochs=20001, batch_size=48, sample_interval=1000)
print("-----------------------------------------------------------------")